In [11]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os

def process_multiple_inputs(input_file_output_folder_pairs):
    """
    Processes multiple input file pairs, computes the classification report and confusion matrix,
    and saves the outputs (confusion matrix image and classification report CSV) to the respective output folders.

    Args:
        input_file_output_folder_pairs (list of tuples): A list where each tuple contains:
            - First element: List of two file paths [predictions_file, labels_file].
            - Second element: Output folder path for saving results.

    Returns:
        None
    """
    # Define the column headers and label mapping
    columns = ['Mild Pain', 'No Pain', 'Pain']
    label_mapping = {0: 'No Pain', 1: 'Mild Pain', 2: 'Obvious Pain'}

    for file_pair in input_file_output_folder_pairs:
        # Unpack each tuple
        input_files, output_folder = file_pair
        predictions_file = input_files[0]
        labels_file = input_files[1]

        # Ensure the output folder exists
        os.makedirs(output_folder, exist_ok=True)

        # Read and process labels
        with open(labels_file, 'r') as lab:
            labels = lab.readlines()
            labels = [x.strip() for x in labels]
            labels = [x.split(' ') for x in labels]
            labels = pd.DataFrame(labels, columns=columns)
            labels = labels.astype(int).idxmax(axis=1)  # Convert one-hot to class labels

        # Read and process predictions
        with open(predictions_file, 'r') as pred:
            predictions = pred.readlines()
            predictions = [x.strip() for x in predictions]
            predictions = [x.split(' ') for x in predictions]
            predictions = pd.DataFrame(predictions, columns=columns)
            predictions = predictions.astype(int).idxmax(axis=1)  # Convert one-hot to class labels

        # Generate and save classification report
        report = classification_report(labels, predictions, target_names=label_mapping.values(), output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_csv_path = os.path.join(output_folder, 'classification_report.csv')
        report_df.to_csv(report_csv_path)
        print(f"Classification report saved at: {report_csv_path}")

        # Generate and save confusion matrix
        cm = confusion_matrix(labels, predictions)
        plt.figure(figsize=(7, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='viridis',
                    xticklabels=label_mapping.values(), yticklabels=label_mapping.values(), annot_kws={"size": 14})
        # Customizing visualization
        plt.xlabel("Predicted", fontsize=16, fontweight='bold')
        plt.ylabel("Actual", fontsize=16, fontweight='bold')
        plt.xticks(fontsize=14)  # Set x-ticklabel font size
        plt.yticks(fontsize=14)  # Set y-ticklabel font size
        # No title
        confusion_matrix_path = os.path.join(output_folder, 'confusion_matrix.png')
        plt.savefig(confusion_matrix_path, bbox_inches='tight', pad_inches=0.1)
        plt.close()  # Close the plot to avoid memory issues
        print(f"Confusion matrix saved at: {confusion_matrix_path}")
# Example list of input file pairs
input_file_output_folder_pairs = [
    (['pain_predictions_only_bb.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'only backbone'),
    (['pain_predictions_no_gnn.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'no gnn'),
    (['pain_predictions.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'full'),
    (['full gr.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'full + graph representation'),
    (['full gr no sft.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'full no sft'),
]

# Call the function
process_multiple_inputs(input_file_output_folder_pairs)

Classification report saved at: only backbone\classification_report.csv
Confusion matrix saved at: only backbone\confusion_matrix.png
Classification report saved at: no gnn\classification_report.csv
Confusion matrix saved at: no gnn\confusion_matrix.png
Classification report saved at: full\classification_report.csv
Confusion matrix saved at: full\confusion_matrix.png
Classification report saved at: full + graph representation\classification_report.csv
Confusion matrix saved at: full + graph representation\confusion_matrix.png
Classification report saved at: full no sft\classification_report.csv
Confusion matrix saved at: full no sft\confusion_matrix.png


In [32]:
import pandas as pd
import os

# Folder names and abbreviated model names
folders = {'full + graph representation': 'Full', 'full no sft': 'W/o SFT', 'full': 'W/o graph rep.', 'no gnn': 'W/o GNN','only backbone': 'Only ResNet' }
categories = ['No Pain', 'Mild', 'Obvious']
# Initialize an empty dictionary to store the metrics
model_metrics = {}

# Loop through each folder to extract required metrics (F1-score, recall, and precision)
for folder, abbrev in folders.items():
    report_path = os.path.join(folder, 'classification_report.csv')
    if os.path.exists(report_path):
        report = pd.read_csv(report_path, index_col=0)

        # Get F1-scores, recall, and precision for each class
        f1_scores = report.loc[['No Pain', 'Mild Pain', 'Obvious Pain'], 'f1-score'].values
        recalls = report.loc[['No Pain', 'Mild Pain', 'Obvious Pain'], 'recall'].values
        precisions = report.loc[['No Pain', 'Mild Pain', 'Obvious Pain'], 'precision'].values

        # Save F1, recall, and precision for the current model
        model_metrics[abbrev] = {'f1': f1_scores, 'recall': recalls, 'precision': precisions}

# Calculate min and max for each category across all metrics
min_max_values = {}
for metric in ['f1', 'recall', 'precision']:
    min_max_values[metric] = {i: (float('inf'), float('-inf')) for i in range(len(categories))}

for model in model_metrics.values():
    for metric in min_max_values.keys():
        for i, value in enumerate(model[metric]):
            current_min, current_max = min_max_values[metric][i]
            min_max_values[metric][i] = (min(current_min, value), max(current_max, value))

# Helper function to apply color based on category-specific min-max value
def color_cell(value, min_val, max_val):
    if max_val == min_val:  # Handle case when all values are the same
        normalized_value = 0.5  # Assign a neutral midpoint
    else:
        normalized_value = (value - min_val) / (max_val - min_val)
    green_intensity = int(155 + 100 * normalized_value)
    red_intensity = int(255 - 100 * normalized_value)
    return f"\\cellcolor[RGB]{{{red_intensity},{green_intensity},155}}{value:.1f}"

# Prepare the values to be inserted into the LaTeX table
table_data = []
metrics = ['F1', 'Recall', 'Precision']

# Loop through the models and metrics to create rows
for abbrev in folders.values():
    for metric_idx, metric in enumerate(metrics):
        row = [abbrev if metric_idx == 0 else "", metric]  # Add model name for the first metric row only
        for i in range(len(model_metrics[abbrev][metric.lower()])):
            value = float(model_metrics[abbrev][metric.lower()][i]) * 100
            min_val, max_val = min_max_values[metric.lower()][i]
            row.append(color_cell(value, min_val * 100, max_val * 100))  # Apply cell coloring
        table_data.append(row)

# Construct the LaTeX table
latex_table = "\\begin{table}[htbp]\n\\centering\n\\begin{tabular}{l|l|" + "|".join(["c"] * len(categories)) + "}\n"
latex_table += "\\hline\n"
latex_table += "\\textbf{Model} & \\textbf{Metric} & " + " & ".join([f"\\textbf{{{category}}}" for category in categories]) + " \\\\\n"
latex_table += "\\hline\n"

# Add rows to the LaTeX table
for row in table_data:
    latex_table += f"\\multirow{{3}}{{*}}{{{row[0]}}} & {row[1]} & " if row[0] != "" else f" & {row[1]} & "
    latex_table += " & ".join(row[2:]) + " \\\\\n"
    if row[1] == "Precision":  # Add \hline after the last row of each model
        latex_table += "\\hline\n"

latex_table += "\\end{tabular}\n"
latex_table += "\\caption{Three-category classification results with separate columns for models and metrics. F1, recall, and precision are shown for No Pain, Mild Pain, and Pain categories.}\n"
latex_table += "\\label{tab:all_models_results}\n"
latex_table += "\\end{table}"

print(latex_table)

\begin{table}[htbp]
\centering
\begin{tabular}{l|l|c|c|c}
\hline
\textbf{Model} & \textbf{Metric} & \textbf{No Pain} & \textbf{Mild} & \textbf{Obvious} \\
\hline
\multirow{3}{*}{Full} & F1 & \cellcolor[RGB]{155,255,155}93.1 & \cellcolor[RGB]{155,255,155}51.2 & \cellcolor[RGB]{165,244,155}54.3 \\
 & Recall & \cellcolor[RGB]{155,255,155}92.9 & \cellcolor[RGB]{184,225,155}52.5 & \cellcolor[RGB]{185,224,155}51.0 \\
 & Precision & \cellcolor[RGB]{192,217,155}93.4 & \cellcolor[RGB]{155,255,155}50.0 & \cellcolor[RGB]{166,243,155}58.1 \\
\hline
\multirow{3}{*}{W/o SFT} & F1 & \cellcolor[RGB]{164,245,155}90.1 & \cellcolor[RGB]{163,246,155}47.3 & \cellcolor[RGB]{155,255,155}60.4 \\
 & Recall & \cellcolor[RGB]{170,239,155}86.1 & \cellcolor[RGB]{169,240,155}63.3 & \cellcolor[RGB]{176,233,155}56.1 \\
 & Precision & \cellcolor[RGB]{173,236,155}94.6 & \cellcolor[RGB]{183,226,155}37.8 & \cellcolor[RGB]{155,255,155}65.5 \\
\hline
\multirow{3}{*}{W/o graph rep.} & F1 & \cellcolor[RGB]{173,236,155}87.7 &